# Courtright PBR Extraction

Courtright Reservior: https://www.google.com/maps/place/Courtright+Reservoir/@37.1016292,-119.0217487,13z/data=!4m6!3m5!1s0x809596e61f5a4355:0xf1aa6387a6022d1e!8m2!3d37.1019078!4d-118.9732411!16zL20vMGQ3cjhn?entry=ttu 

## 1. Organize data
Create a folder under `semantic_SfM/data` and organize your data following the structures below. In this case, I use Agisoft because the camera intrinsic parameter $c_y$ estimated from WebODM has an error of 40 pixels.

Agisoft:
```
semantic_SfM/data
    ├── courtright
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── agisoft_cameras.xml
        │       └── agisoft_model.las   
        ├── segmentations
        └── associations

```

## 2. Estimate memory usage

In [1]:
from ssfm.memory_calculator import memory_calculator

In [2]:
# pointcloud file
las_file = "../data/courtright/SfM_products/agisoft_model.las"
# image file sample
image_file = "../data/courtright/DJI_photos/DJI_0576.JPG"
# number of images
num_images = 294
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+----------------------+
|              Memory Type               | Memory Required (GB) |
+----------------------------------------+----------------------+
|      Segmentation for each image       | 0.037181854248046875 |
| Pixel2point association for each image | 0.07436370849609375  |
| Point2pixel association for each image |  0.308795440942049   |
|                                        |                      |
|      Segmentation for all images       |  10.931465148925781  |
| Pixel2point association for all images |  21.862930297851562  |
| Point2pixel association for all images |  90.78585963696241   |
|          pc_segmentation_ids           |  1.5439772047102451  |
|         pc_segmentation_probs          |  1.5439772047102451  |
|          keyimage_association          |  22.696464909240603  |
|                 Total                  |  149.36467440240085  |
+----------------------------------------+----------------------+


## 3. Create 2D Segmentation using SAM

In [3]:
from ssfm.image_segmentation import ImageSegmentation
import os

In [14]:
sam_params = {}
sam_params['model_name'] = 'sam'
sam_params['model_path'] = '../semantic_SfM/sam/sam_vit_h_4b8939.pth'
sam_params['model_type'] = 'vit_h'
sam_params['device'] = 'cuda:1'
sam_params['points_per_side'] = 64
sam_params['pred_iou_thresh'] = 0.96
sam_params['stability_score_thresh'] = 0.92

In [15]:
image_segmentor = ImageSegmentation(sam_params)   
image_folder_path = '../data/courtright/DJI_photos'
segmentation_folder_path = '../data/courtright/segmentations'
image_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
image_segmentor.batch_predict(image_paths, segmentation_folder_path, maximum_size=1000, save_overlap=True)

Processing image 1/294.
Processing image 2/294.
Processing image 3/294.
Processing image 4/294.
Processing image 5/294.
Processing image 6/294.
Processing image 7/294.
Processing image 8/294.
Processing image 9/294.
Processing image 10/294.
Processing image 11/294.
Processing image 12/294.
Processing image 13/294.
Processing image 14/294.
Processing image 15/294.
Processing image 16/294.
Processing image 17/294.
Processing image 18/294.
Processing image 19/294.
Processing image 20/294.
Processing image 21/294.
Processing image 22/294.
Processing image 23/294.
Processing image 24/294.
Processing image 25/294.
Processing image 26/294.
Processing image 27/294.
Processing image 28/294.
Processing image 29/294.
Processing image 30/294.
Processing image 31/294.
Processing image 32/294.
Processing image 33/294.
Processing image 34/294.
Processing image 35/294.
Processing image 36/294.
Processing image 37/294.
Processing image 38/294.
Processing image 39/294.
Processing image 40/294.
Processin

## 4. Create projection associations

In [1]:
from ssfm.probabilistic_projection import *
import time

In [2]:
pointcloud_projector = PointcloudProjection()
pointcloud_projector.read_camera_parameters('../data/courtright/SfM_products/agisoft_cameras.xml')
pointcloud_projector.read_pointcloud('../data/courtright/SfM_products/agisoft_model.las')

In [3]:
# batch project
image_folder_path = '../data/courtright/DJI_photos'
save_folder_path = '../data/courtright/associations'

image_list = [f for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, save_folder_path, num_workers=8)

Processing frames: 100%|██████████| 294/294 [07:49<00:00,  1.60s/it]


In [4]:
# build keyimage associations
t1 = time.time()
pointcloud_projector.build_associations_keyimage('../data/courtright/associations', '../data/courtright/segmentations')
t2 = time.time()
print('Time for building keyimage associations: ', t2 - t1)

Time for building keyimage associations:  325.84498381614685


## 5. Run object registration

In [5]:
from ssfm.object_registration import *

In [6]:
# Set paths
pointcloud_path = '../data/courtright/SfM_products/agisoft_model.las'
segmentation_folder_path = '../data/courtright/segmentations'
image_folder_path = '../data/courtright/DJI_photos'
association_folder_path = '../data/courtright/associations'


# Create object registration
t1 = time.time()
obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path)
t2 = time.time()
print('Time elapsed for creating object registration: {}'.format(t2-t1))

# Run object registration
#obr.object_registration(iou_threshold=0.5, save_semantics=True)
obr.object_registration(iou_threshold=0.5)

# save semantic point cloud
obr.save_semantic_pointcloud('../data/courtright/semantic_model.las')


Time elapsed for creating object registration: 12.751423358917236
Processing image 1/294: DJI_0576.npy
Processing image 2/294: DJI_0577.npy
Processing image 3/294: DJI_0578.npy
Processing image 4/294: DJI_0579.npy
Processing image 5/294: DJI_0580.npy
Processing image 6/294: DJI_0581.npy
Processing image 7/294: DJI_0582.npy
Processing image 8/294: DJI_0583.npy
Processing image 9/294: DJI_0584.npy
Processing image 10/294: DJI_0585.npy
Processing image 11/294: DJI_0586.npy
Processing image 12/294: DJI_0587.npy
Processing image 13/294: DJI_0588.npy
Processing image 14/294: DJI_0589.npy
Processing image 15/294: DJI_0590.npy
Processing image 16/294: DJI_0591.npy
Processing image 17/294: DJI_0592.npy
Processing image 18/294: DJI_0593.npy
Processing image 19/294: DJI_0594.npy
Processing image 20/294: DJI_0595.npy
Processing image 21/294: DJI_0596.npy
Processing image 22/294: DJI_0597.npy
Processing image 23/294: DJI_0598.npy
Processing image 24/294: DJI_0599.npy
Processing image 25/294: DJI_06

In [8]:
add_semantics_to_pointcloud_flag = True
if add_semantics_to_pointcloud_flag:
    image_id = 269
    semantics_folder_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
    save_las_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
    add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path) 

maximum of semantics:  73
number of unique semantics:  56


In [7]:
from ssfm.post_processing import PostProcessing

In [5]:
semantic_pc_file_path = '../data/courtright/semantic_model.las'
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../data/courtright/semantic_model_shuffled.las'
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  869
